<a href="https://colab.research.google.com/github/eshmaneva/DS-Practical/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Preinstalling the necessary libraries
#Certain versions are required to avoid compatibility issues 

In [2]:
#Mounting google drive to import the data from
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install numpy==1.19.5
!pip install tensorflow==2.7.0
!pip install transformers==4.7.0
!pip install sacremoses==0.0.45

     |████████████████████████████████| 14.8 MB 31.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
tensorflow 2.8.0+zzzcolab20220506162203 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 489.6 MB 23 kB/s 
     |████████████████████████████████| 1.3 MB 60.4 MB/s 
     |████████████████████████████████| 463 kB 72.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0+zzzcolab20220506162203
    Uninstalling tensorflow-2.8.0+zzzcolab20220506162203:
      Successfully uninstalled tensorflow-2.8.0+zzzcolab20220506162203
     |████████████████████████████████| 2.5 MB 34.5 MB/s 
     |█████████████████████████

In [4]:
#Importing necessary classes for classification and summarizaton
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

import pandas as pd
import numpy as np
import nltk
import re

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from six import viewitems
#Importing methods for splitting and shuffling data (as dataset contains no pre-trained data)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#Reading the data as a pandas dataframe
#Katja PATH
df=pd.read_csv("/content/drive/MyDrive/DS Practical/Reviews.csv", engine="python", error_bad_lines=False)

#Valari PATH
#df=pd.read_csv("/content/drive/MyDrive/Reviews.csv", engine="python", error_bad_lines=False)
df.drop(columns=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator', 'Time'],axis=1,inplace=True)
print("Before",len(df))
df = df.dropna()
print("Data size:",len(df))
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Before 568454
Data size: 568427


,Score,Summary,Text
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
#Checking the avaliable GPUs (not necessary, made as a test of the system)

#num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
#print("Num GPUs Available: ", num_gpus_available)
#assert num_gpus_available > 0

In [7]:
'''
#Setting the dataset as a frame (trasforming it from tensor)
df = tfds.as_dataframe(df)
#Preview of the data
df.head()
'''

'\n#Setting the dataset as a frame (trasforming it from tensor)\ndf = tfds.as_dataframe(df)\n#Preview of the data\ndf.head()\n'

In [8]:
#Classifying the data into two classes: positive and negative based on their star rating
df["Sentiment"] = df["Score"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

In [9]:
#df['short_review'] = df['Text'].str.decode("utf-8")

In [10]:
#Creating a sunset of the data with only the required fields
df = df[["Text", "Sentiment"]]

In [11]:
'''
#Dropping last n rows using drop
n = 54975
df.drop(df.tail(n).index,
        inplace = True)
'''

'\n#Dropping last n rows using drop\nn = 54975\ndf.drop(df.tail(n).index,\n        inplace = True)\n'

In [12]:
#Creating a smaller dataset for faster training
df=df.loc[1:10000]
#Dropping the na values
df.dropna()
print("Data size:",len(df))

Data size: 10000


In [13]:
df.head()

,Text,Sentiment
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1
5,I got a wild hair for taffy and ordered this f...,1


In [14]:
#To check how big is the dataset / num of rows
#index = df.index
#number_of_rows = len(index)
#print(number_of_rows)

#Printing the beginning part to see if the data is read correctly
#df.head()

#Printing the beginning part to see if the data is read correctly
#df.tail()

In [15]:
#Testing the labels
reviews = df['Text'].values.tolist()
labels = df['Sentiment'].tolist() #convert to category
#print(reviews[:2])
#print(labels[:2])

In [16]:
#training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=0.33, random_state=42, stratify=labels)
#this is on creating stratifyed sample

In [17]:
#Preprocessing the data using DistilBert for punctuation splitting and wordpieces
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [18]:
#Intiniating the tokenizer for the training data
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

{'input_ids': [[101, 2122, 11772, 2024, 2200, 2204, 999, 999, 1045, 2481, 1005, 1056, 2644, 5983, 2068, 1012, 2027, 2134, 1005, 1056, 2197, 2200, 2146, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 3114, 1045, 2435, 2068, 1018, 3340, 1010, 2612, 1997, 1019, 1010, 2003, 2138, 1045, 2228, 1996, 8641, 2024, 1037, 2210, 2524, 2000, 2330, 1012, 1045, 1005, 1049, 2025, 2469, 2339, 2027, 2342, 2000, 2191, 1037, 14557, 9090, 4524, 2008, 23073, 1010, 2017, 2089, 2342, 2070, 25806, 2006, 2192, 2000, 2330, 2068, 2172, 6082, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [19]:
#Tokenizing the data
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(validation_sentences,
                            truncation=True,
                            padding=True)

In [20]:
#Slicing the dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

print(val_dataset)

<TensorSliceDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.int32)>


In [21]:
#Loading the DistilBert model from transformers
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [22]:
#Defining and fitting the model on the training data
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
callbacks=tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', 
    min_delta=0.0001,
    patience=3,
    mode='auto',
    verbose=2,
    baseline=None
)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16),callbacks=callbacks)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [23]:
'''
import matplotlib.pyplot as plt

plt.title('Loss curves')
plt.plot(model.train_loss_history, '-', label='train')
plt.plot(model.val_loss_history, '-', label='val')
plt.legend(loc='lower right')
plt.xlabel('Iteration')
plt.show()

'''

"\nimport matplotlib.pyplot as plt\n\nplt.title('Loss curves')\nplt.plot(model.train_loss_history, '-', label='train')\nplt.plot(model.val_loss_history, '-', label='val')\nplt.legend(loc='lower right')\nplt.xlabel('Iteration')\nplt.show()\n\n"

In [24]:
#Saving the fitted model
model.save_pretrained("./sentiment")

In [25]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("./sentiment")

Some layers from the model checkpoint at ./sentiment were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./sentiment and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
#Importing the testing data from the T5 generated summaries
import pandas as pd

df1 = pd.read_csv("/content/drive/MyDrive/summary_test.csv", index_col=0, engine="python", error_bad_lines=False)
df1=df1.loc[0:50]
selected_columns = df1[["Summary","Text","Generated_summary"]]
df = selected_columns.copy()
df = df.dropna()


df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,Summary,Text,Generated_summary
0,Repeated deliveries of broken jars. Great pro...,liked getting one twin 10-month olds interesti...,I really like the green ingredients
1,"Good flavor, too runny though","baby loved one, progressed cut one runny move ...",Great for a little bit of water
2,12% Protein and 50% Vitamin A,earth's best turkey vegetable dinner hits spot...,Delicious!
3,Baby Lilly says 2 thumbs up!,little girl can't get enough! like organic con...,I love organic!
4,Earth's Best Rice & Lentil dinner,daughter's favorite first foremost. love get l...,Love lentils


In [27]:
for i in range(0, len(df)):
  df['Sentiment_text'] = i 
  df['Sentiment_summary'] = i

In [28]:
#Classifying the text into sentiment classes
for i in range(0,len(df)):
  
  predict_input_text = tokenizer.encode(df['Text'][i],
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
  tf_output_text = loaded_model.predict(predict_input_text)[0]
  tf_prediction_text = tf.nn.softmax(tf_output_text, axis=1)
  labels = ['Negative','Positive']
  label_text = tf.argmax(tf_prediction_text, axis=1)
  label_text = label_text.numpy()
  df["Sentiment_text"][i] = (labels[label_text[0]])

  
#df = df.append(data, columns = "Sentiment")
#print(df['Text'], df['Sentiment_text'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [29]:
#Classifying the generated summaries into sentiment classes
for i in range(0, len(df)):
  
  predict_input_sum = tokenizer.encode(df['Generated_summary'][i],
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
  tf_output_sum = loaded_model.predict(predict_input_sum)[0]
  tf_prediction_sum = tf.nn.softmax(tf_output_sum, axis=1)
  #labels = ['Negative','Positive']
  label_sum = tf.argmax(tf_prediction_sum, axis=1)
  label_sum = label_sum.numpy()
  df["Sentiment_summary"][i] = (labels[label_sum[0]])


#print(df['Generated_summary'], df['Sentiment_summary'])
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Summary,Text,Generated_summary,Sentiment_text,Sentiment_summary
0,Repeated deliveries of broken jars. Great pro...,liked getting one twin 10-month olds interesti...,I really like the green ingredients,Positive,Positive
1,"Good flavor, too runny though","baby loved one, progressed cut one runny move ...",Great for a little bit of water,Positive,Positive
2,12% Protein and 50% Vitamin A,earth's best turkey vegetable dinner hits spot...,Delicious!,Positive,Positive
3,Baby Lilly says 2 thumbs up!,little girl can't get enough! like organic con...,I love organic!,Positive,Positive
4,Earth's Best Rice & Lentil dinner,daughter's favorite first foremost. love get l...,Love lentils,Positive,Positive


In [30]:
#Calculating the error by checking the difference in classes for generated summary and text input
tag = 0
for i in range(0, len(df)):
  if (df['Sentiment_text'][i] != df['Sentiment_summary'][i]):
    tag = tag + 1

Error = tag/len(df)
print(Error)

0.2
